<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">Visualización de Datos · PEC2</p>
<p style="margin: 0; text-align:right;">Máster universitario en Ciencia de datos (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>


# PEC 2: Estudios de técnicas de Visualización de Datos

A lo largo de esta práctica vamos a implementar varios modelos de redes neuronales recurrentes para la detección, clasificación y localización de errores en datos temporales producidos por varios sensores. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
paro = pd.read_csv('./data/tasa_paro.csv', sep=';')
activos = pd.read_csv('./data/activos.csv', sep=';')

In [3]:
paro.head()
activos.head()

,Sexo,Comunidades y Ciudades Autónomas,Edad,Periodo,Total
0,Ambos sexos,Total Nacional,Total,2023T3,"24.121,00"
1,Ambos sexos,Total Nacional,Total,2023T2,"23.819,20"
2,Ambos sexos,Total Nacional,Total,2023T1,"23.580,50"
3,Ambos sexos,Total Nacional,Total,2022T4,"23.487,80"
4,Ambos sexos,Total Nacional,Total,2022T3,"23.525,90"


In [4]:
subset_paro = paro[(paro['Edad'] == 'Total') & (paro['Comunidades y Ciudades Autónomas'] != 'Total Nacional') & 
                   (paro['Sexo'] == 'Ambos sexos')]

subset_activos = activos[(activos['Edad'] == 'Total') & (activos['Comunidades y Ciudades Autónomas'] != 'Total Nacional') & 
                   (activos['Sexo'] == 'Ambos sexos')]

In [5]:
subset_paro.loc[:, 'Periodo'] = subset_paro['Periodo'].str.extract(r'(\d{4})').astype(int)
subset_paro['Total'] = subset_paro['Total'].str.replace(',', '.').astype(float)

subset_activos.loc[:, 'Periodo'] = subset_activos['Periodo'].str.extract(r'(\d{4})').astype(int)
subset_activos['Total'] = subset_activos['Total'].str.replace('.', '').str.replace(',', '.').astype(float)

C:\Users\alexo\AppData\Local\Temp\ipykernel_17184\621662536.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_paro.loc[:, 'Periodo'] = subset_paro['Periodo'].str.extract(r'(\d{4})').astype(int)
C:\Users\alexo\AppData\Local\Temp\ipykernel_17184\621662536.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_paro['Total'] = subset_paro['Total'].str.replace(',', '.').astype(float)
C:\Users\alexo\AppData\Local\Temp\ipykernel_17184\621662536.py:4: SettingWithCopyWarning: 
A value is trying to be se

In [6]:
df_anual_paro = subset_paro.groupby(['Comunidades y Ciudades Autónomas', 'Periodo'])['Total'].mean().reset_index()
df_anual_activos = subset_activos.groupby(['Comunidades y Ciudades Autónomas', 'Periodo'])['Total'].mean().reset_index()

In [7]:
df_anual_activos.head()

,Comunidades y Ciudades Autónomas,Periodo,Total
0,01 Andalucía,2002,3165.550
1,01 Andalucía,2003,3260.225
2,01 Andalucía,2004,3360.975
3,01 Andalucía,2005,3480.250
4,01 Andalucía,2006,3591.850


In [8]:
df_anual_paro.head()

,Comunidades y Ciudades Autónomas,Periodo,Total
0,01 Andalucía,2002,19.4950
1,01 Andalucía,2003,18.4500
2,01 Andalucía,2004,16.9925
3,01 Andalucía,2005,13.8250
4,01 Andalucía,2006,12.6225


In [10]:
df_anual_activos.rename(columns={'Total': 'Activos'}, inplace=True)
df_anual_paro.rename(columns={'Total': 'Paro'}, inplace=True)

df_total = pd.merge(df_anual_activos, df_anual_paro, 
                    on=['Comunidades y Ciudades Autónomas', 'Periodo'], how='inner')

In [12]:
df_total['Comunidades y Ciudades Autónomas'] = df_total['Comunidades y Ciudades Autónomas'].str.replace(r'^\d+\s*', '', regex=True)

In [13]:
df_total.head()

,Comunidades y Ciudades Autónomas,Periodo,Activos,Paro
0,Andalucía,2002,3165.550,19.4950
1,Andalucía,2003,3260.225,18.4500
2,Andalucía,2004,3360.975,16.9925
3,Andalucía,2005,3480.250,13.8250
4,Andalucía,2006,3591.850,12.6225


In [17]:
df_total.to_csv('./data/paro_españa.csv', index=False, encoding='utf-8')